### Week 3 Part 1

### Import nesseccery libraries

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install beautifulsoup4 --yes
from bs4 import BeautifulSoup

!conda install xml
import xml

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

### Import data from wekipedia using BeautifulSoup

#get url
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
r = requests.get(url).text
soup = BeautifulSoup(r)

### Find table and scrapt data

table = soup.find('table')
fields = table.find_all('td')

postcode = []
borough = []
neighbourhood = []

for i in range(0, len(fields), 3):
    postcode.append(fields[i].text.strip())
    borough.append(fields[i+1].text.strip())
    neighbourhood.append(fields[i+2].text.strip())
    
df_post = pd.DataFrame(data=[postcode, borough, neighbourhood])
df_post

df_post = df_post.T
df_post.columns = ['PostalCode', 'Borough', 'Neighborhood']
df_post.head()

### Repalce "Not assigned" with NaN in Borough column and then drop it

df_post['Borough'].replace('Not assigned', np.nan, inplace=True)
df_post.head()

df_post.dropna(subset=['Borough'], axis=0, inplace=True)
df_post.head()

### Group neighborhood

df_post_group = df_post.groupby(['PostalCode', 'Borough'])['Neighborhood'].apply('.'.join).to_frame()
df_post_group.head()

df_post_group.reset_index(inplace=True)
df_post_group.head()

### There is no "Not assigned" in Neighborhood column

df_post_group[df_post_group['Neighborhood'] == 'Not assigned']

df_post_group.shape

### Week 3 Part 2

In [32]:
geodata = pd.read_csv('http://cocl.us/Geospatial_data')
geodata.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### merge geodata 

In [35]:
df = pd.merge(df_post_group, geodata, left_on='PostalCode', right_on='Postal Code')
df.drop('Postal Code', axis=1, inplace=True)
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,Malvern / Rouge,43.806686,-79.194353
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,43.784535,-79.160497
2,M1E,Scarborough,Guildwood / Morningside / West Hill,43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
